In [1]:
import os
import random
from PIL import Image
import numpy as np
import scipy.misc

In [2]:
def modify_bbox(x1, y1, x2, y2, height, width, imagePath):
    image = Image.open(imagePath)
    (currw, currh) = image.size
    if currw >= currh:
        resize = float(width) / float(currw)
    else:
        resize = float(height) / float(currh)
    x1 = int(resize * x1)
    y1 = int(resize * y1)
    x2 = int(resize * x2)
    y2 = int(resize * y2)
    if currw >= currh:
        y1 = int(y1 + 0.5 * (height - resize * currh))
        y2 = int(y2 + 0.5 * (height - resize * currh))
    else:
        x1 = int(x1 + 0.5 * (width - resize * currw))
        x2 = int(x2 + 0.5 * (width - resize * currw))
    print([(x1,y1),(x2,y2)])
    return([(x1,y1),(x2,y2)])

In [16]:
def draw_bounding_boxes(box, img):

    img = img.convert("RGBA")

    for x in range(box[0][0], box[1][0] + 1):
        for y in range(box[0][1], box[1][1] + 1):
            rgb = img.getpixel((x,y))
            rgb = (rgb[0], rgb[1], rgb[2], 255)
            rgb = (int(0.8*rgb[0]), int(0.8*rgb[1]), int(0.8*rgb[2]), 255)
            img.putpixel((x,y), rgb)
    for x in range(box[0][0], box[1][0] + 1):
        img.putpixel((x,box[0][1]), (0,0,255,255))
        img.putpixel((x,box[1][1]), (0,0,255,255))
    for y in range(box[0][1], box[1][1] + 1):
        img.putpixel((box[0][0],y), (0,0,255,255))
        img.putpixel((box[1][0],y), (0,0,255,255))
    img.show()

In [4]:
def processImage(imagePath):
    image = Image.open(imagePath)
    (currw, currh) = image.size
    width = 299
    height = 299
    if currw >= currh:
        image = image.resize((width, int(float(currh) * float(width) / float(currw))))
    else:
        image = image.resize((int(float(currw) * float(height) / float(currh)), height))
    background = Image.new('RGB', (width, height), (0, 0, 0))
    background.paste(
        image, (int((width - image.size[0]) / 2), int((height - image.size[1]) / 2))
    )
    return background

In [17]:
img = Image.open('/Users/anshulramachandran/Documents/Year3 Q3/CS148/CUB_200_2011/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg')
draw_bounding_boxes([(60, 27), (60+325, 27+304)], img)

In [32]:
img = Image.open('/Users/anshulramachandran/Documents/Year3 Q3/CS148/CUB_200_2011/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg')
img = processImage('/Users/anshulramachandran/Documents/Year3 Q3/CS148/CUB_200_2011/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg')
bx = modify_bbox(60, 27, 60+325, 27+304, 299, 299, '/Users/anshulramachandran/Documents/Year3 Q3/CS148/CUB_200_2011/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg')
draw_bounding_boxes(bx, img)

[(35, 16), (230, 197)]


In [38]:
img = Image.open('/Users/anshulramachandran/Documents/Year3 Q3/CS148/CUB_200_2011/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg')
img = processImage('/Users/anshulramachandran/Documents/Year3 Q3/CS148/CUB_200_2011/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg')
bx = modify_bbox(60, 27, 60+325, 27+304, 299, 299, '/Users/anshulramachandran/Documents/Year3 Q3/CS148/CUB_200_2011/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg')
draw_bounding_boxes(bx, img)

[(35, 65), (230, 246)]


In [2]:
height = 299.0
width = 299.0

aspect_ratios = [[1.0, 1.0], [1.0, 0.67], [0.67, 1.0], [0.8, 0.6], [0.6, 0.8], [1.0, 0.75], [0.75, 1.0], [1.0, 0.6], [0.6, 1.0], [1.0, 0.4], [0.4, 1.0]]

def make_prior_box(x1, y1, x2, y2, ar):
    return [x1 + 0.5*(1.0-ar[0])*(x2 - x1), 
            y1 + 0.5*(1.0-ar[1])*(y2 - y1), 
            x2 - 0.5*(1.0-ar[0])*(x2 - x1), 
            y2 - 0.5*(1.0-ar[1])*(y2 - y1)]

make_prior_box(0.0, 0.0, 100.0, 100.0, [0.8, 0.6])

[9.999999999999998, 20.0, 90.0, 80.0]

In [9]:
def get_pboxes(height, width):

    pboxes = []

    for ar in aspect_ratios:
        for i in range(8):
            for j in range(8):
                pboxes.append(make_prior_box(i*height/9.0, j*width/9.0, (i+1)*height/9.0, (j+1)*width/9.0, ar))
            
    for ar in aspect_ratios:
        for i in range(6):
            for j in range(6):
                pboxes.append(make_prior_box(i*height/7.0, j*width/7.0, (i+1)*height/7.0, (j+1)*width/7.0, ar))
            
    for ar in aspect_ratios:
        for i in range(4):
            for j in range(4):
                pboxes.append(make_prior_box(i*height/5.0, j*width/5.0, (i+1)*height/5.0, (j+1)*width/5.0, ar))

    for ar in aspect_ratios:
        for i in range(3):
            for j in range(3):
                pboxes.append(make_prior_box(i*height/4.0, j*width/4.0, (i+1)*height/4.0, (j+1)*width/4.0, ar))
            
    for ar in aspect_ratios:
        for i in range(2):
            for j in range(2):
                pboxes.append(make_prior_box(i*height/3.0, j*width/3.0, (i+1)*height/3.0, (j+1)*width/3.0, ar))

    pboxes.append([0.1*height, 0.1*width, 0.9*height, 0.9*width])
    
    return np.asarray(pboxes)



In [10]:
get_pboxes(299.0, 299.0).shape

(1420, 4)